In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import tensorflow as tf
import keras
import sys
from tensorflow.keras.layers import Layer, Dense, LSTM, GRU, Dropout, Input, Concatenate, Bidirectional, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import keras_tuner as kt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split

# For reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Setting directory
os.chdir('/Users/manotas/Documents/GitHub-Repos/CFForecast_Energy_OptionValuation')

In [9]:
data = pd.read_csv('data/processed/TopRevenueMNC.csv')

In [3]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print()
print("GPU is", "AVAILABLE" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.14.0
Keras Version: 2.14.0

Python 3.9.13 (main, Oct 20 2023, 23:53:00) 
[Clang 15.0.0 (clang-1500.0.40.1)]

GPU is AVAILABLE


In [58]:
# Melt the dataframe to long format
value_vars = [col for col in data.columns if any(year in col for year in ['2023', '2022', '2021', '2020', '2019'])]
id_vars = [col for col in data.columns if col not in value_vars]

# Create a new DataFrame by melting
melted_data = pd.melt(data, id_vars=id_vars, value_vars=value_vars, 
                      var_name='metric_year', value_name='value')

# Extract metric and year from 'metric_year'
melted_data['metric'] = melted_data['metric_year'].apply(lambda x: '_'.join(x.split('_')[:-1]))
melted_data['year'] = melted_data['metric_year'].apply(lambda x: x.split('_')[-1]).astype(int)

# Pivot the table to have years as rows
processed_data = melted_data.pivot_table(index=id_vars + ['year'], columns='metric', values='value').reset_index()

# Initialize the scaler and encoder
scaler = StandardScaler()

# Scale the numerical columns
numerical_columns = ['operating_revenue', 'net_income', 'total_assets', 
                     'profit_margin', 'solvency_ratio', 'cogs', 'ebitda', 'cash', 'cash_flow']
processed_data[numerical_columns] = scaler.fit_transform(processed_data[numerical_columns])

# Encode categorical columns
processed_data = processed_data.drop(columns=['quoted','country','industry'])

processed_data.head(10)

metric,company,year,cash,cash_flow,cogs,ebitda,net_income,operating_revenue,profit_margin,solvency_ratio,total_assets
0,A2A S.P.A.,2019,-0.186611,-0.113742,0.050296,-0.107829,-0.050281,-0.083231,-0.053813,-0.212344,-0.139042
1,A2A S.P.A.,2020,0.041915,-0.092808,0.051733,-0.096022,-0.047253,-0.078214,-0.120052,-0.227164,-0.092646
2,A2A S.P.A.,2021,-0.003531,-0.057679,-0.123806,-0.077898,-0.008334,0.043173,-0.224178,-0.612303,-0.007939
3,A2A S.P.A.,2022,0.500703,-0.081176,-0.571526,-0.079468,-0.053195,0.346078,-0.335542,-0.730119,0.029664
4,A2A S.P.A.,2023,0.211043,-0.010680,-0.221091,-0.005285,0.040992,0.127237,-0.173080,-0.547266,-0.002035
5,AB IGNITIS GROUP,2019,-0.289456,-0.251609,0.242775,-0.254855,-0.167304,-0.265420,-0.168918,0.108034,-0.271846
6,AB IGNITIS GROUP,2020,-0.089401,-0.225434,0.240854,-0.234367,-0.121560,-0.258577,0.431160,0.268695,-0.252706
7,AB IGNITIS GROUP,2021,-0.180229,-0.227265,0.213895,-0.238076,-0.132620,-0.241792,0.005308,0.160182,-0.252650
8,AB IGNITIS GROUP,2022,-0.110027,-0.205151,0.125463,-0.212509,-0.089168,-0.175683,-0.068472,0.035192,-0.239964
9,AB IGNITIS GROUP,2023,-0.265597,-0.196564,0.199222,-0.214765,-0.076354,-0.224180,0.305136,0.146977,-0.237257


In [59]:
# Assuming `processed_data` is the DataFrame
companies = processed_data['company'].unique()
X_all, y_all = [], []

for company in companies:
    company_data = processed_data[processed_data['company'] == company].sort_values(by='year')
    X_company, y_company = [], []
    for i in range(len(company_data) - 1):
        X_company.append(company_data.iloc[i, 3:].values)  # All columns except 'company', 'quoted', 'country', 'industry', 'year'
        y_company.append(company_data.iloc[i + 1]['cash_flow'])  # Target is cash_flow at t
    
    X_all.append(np.array(X_company))
    y_all.append(np.array(y_company))

# Convert to numpy arrays and change the dtype to float32
X_all = np.array(X_all, dtype=np.float32)
y_all = np.array(y_all, dtype=np.float32)

# Verify the shapes
print(f'Shape of X_all: {X_all.shape}')
print(f'Shape of y_all: {y_all.shape}')

Shape of X_all: (118, 4, 8)
Shape of y_all: (118, 4)


In [60]:
class Time2Vec(tf.keras.layers.Layer):
    def __init__(self, kernel_size=1, **kwargs):
        self.kernel_size = kernel_size
        super(Time2Vec, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='W', shape=(input_shape[-1], self.kernel_size),
                                 initializer='uniform', trainable=True)
        self.B = self.add_weight(name='B', shape=(1, self.kernel_size),
                                 initializer='uniform', trainable=True)
        self.w = self.add_weight(name='w', shape=(input_shape[-1],),
                                 initializer='uniform', trainable=True)
        self.b = self.add_weight(name='b', shape=(input_shape[-1],),
                                 initializer='uniform', trainable=True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs):
        bias = self.w * inputs + self.b
        dp = tf.matmul(inputs, self.W) + self.B
        wgts = tf.math.sin(dp)
        return tf.concat([wgts, bias], -1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[1], input_shape[2] * 2

def build_model(hp):
    inputs = Input(shape=(X_all.shape[1], X_all.shape[2]))
    time2vec = Time2Vec(kernel_size=hp.Int('t2v_kernel_size', min_value=1, max_value=5))(inputs)
    
    x = time2vec
    
    for i in range(hp.Int('num_layers', 1, 3)):
        if hp.Choice('rnn_type', ['LSTM', 'GRU']) == 'LSTM':
            x = Bidirectional(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), 
                                   return_sequences=True))(x)
        else:
            x = Bidirectional(GRU(units=hp.Int('units', min_value=32, max_value=128, step=32), 
                                  return_sequences=True))(x)
        if hp.Boolean('dropout'):
            x = Dropout(rate=0.2)(x)

    # Adding Attention Mechanism
    attention = Attention()([x, x])
    
    # Reduce to single output
    x = Dense(1)(attention)
    
    model = Model(inputs, x)
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='mse', 
                  metrics=['mae'])
    return model

In [64]:
tuner = kt.BayesianOptimization(build_model,
                                objective='val_mae',
                                max_trials=100,
                                directory='models/bayesian_optimization',
                                project_name='time2vec_rnn')

# Splitting the data into train and test sets
split_idx = int(0.8 * len(X_all))
X_train_np, X_test_np = X_all[:split_idx], X_all[split_idx:]
y_train_np, y_test_np = y_all[:split_idx], y_all[split_idx:]

tuner.search(X_train_np, y_train_np, epochs=100, validation_data=(X_test_np, y_test_np))

Trial 100 Complete [00h 00m 55s]
val_mae: 0.13455837965011597

Best val_mae So Far: 0.09569685906171799
Total elapsed time: 01h 08m 27s


In [67]:
# Retrieving the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Verifying the best hyperparameters
tuner.get_best_hyperparameters()[0].values

{'t2v_kernel_size': 2,
 'num_layers': 2,
 'rnn_type': 'LSTM',
 'units': 96,
 'dropout': False,
 'learning_rate': 0.001}

In [68]:
# Evaluate the best model
best_model.evaluate(X_test_np, y_test_np)

2024-07-03 13:31:35.600657: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


1/1 [==============================] - 8s 8s/step - loss: 0.0498 - mae: 0.0957


[0.049753278493881226, 0.09569685906171799]

In [69]:
# Storing the best model
best_model.save('models/time2vec_rnn_best_model')

INFO:tensorflow:Assets written to: models/time2vec_rnn_best_model/assets


INFO:tensorflow:Assets written to: models/time2vec_rnn_best_model/assets


In [70]:
y_pred_train = np.squeeze(best_model.predict(X_train_np), axis=-1)
y_pred_test = np.squeeze(best_model.predict(X_test_np), axis=-1)

2024-07-03 13:32:01.490070: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


1/1 [==============================] - 2s 2s/step


In [71]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_metrics(y_actual, predictions):
    '''
    A function that calculates and prints the RMSE, MAE, sMAPE and R2 score of a given actual and predicted series.
    A caveat of sMAPE, we measure it as outlined by Makridakis and Hibon (2000). Further information can be found in:
    https://robjhyndman.com/hyndsight/smape/. As well as both sources defined in the references folder.
    '''
    # Calculate Mean Squared Error and Root Mean Squared Error
    mse = mean_squared_error(y_actual, predictions)
    rmse = np.sqrt(mse)
    
    # Calculate Mean Absolute Error
    mae = mean_absolute_error(y_actual, predictions)
    
    # Calculate Symmetric Mean Absolute Percentage Error (SMAPE)
    smape = np.mean(2.0 * np.abs(y_actual - predictions) / ((np.abs(y_actual) + np.abs(predictions)) + 1e-10)) * 100
    
    # Calculate R-squared
    r2 = r2_score(y_actual, predictions)

    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')
    print(f'sMAPE(0-200): {smape}%')
    print(f'R-squared: {r2}')

In [72]:
print('Best RNN model - Train metrics:')
calculate_metrics(y_train_np, y_pred_train)
print('\n Best RNN model - Test metrics:')
calculate_metrics(y_test_np, y_pred_test)

Best RNN model - Train metrics:
RMSE: 0.22897104918956757
MAE: 0.06671774387359619
sMAPE(0-200): 18.333913385868073%
R-squared: 0.8091707007963322

 Best RNN model - Test metrics:
RMSE: 0.223054438829422
MAE: 0.09569685161113739
sMAPE(0-200): 37.482818961143494%
R-squared: 0.9872139223029276
